In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [47]:
import pandas as pd
import re
import math
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import preprocessing
import numpy as np
import pandas as pd
import re as re
import lxml.html
import lxml.html.clean
from nltk.tokenize import RegexpTokenizer
from string import digits
import nltk
from nltk.corpus import stopwords
import pickle
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
import scipy
from scipy.sparse import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

nltk.download('words')
nltk.download('stopwords')
from sklearn.metrics import mean_squared_error
# from google.colab import drive
# drive.mount('/content/drive')

token = RegexpTokenizer("[\w']+")
sw = stopwords.words('english')
tokenizer = (nltk.corpus.words.words())

ModuleNotFoundError: No module named 'joblib'

In [3]:
train_data = pd.read_csv("dataset/training/data_train.csv", names=['country', 'sku_id', 'title', 'category_lvl_1', 'category_lvl_2', 'category_lvl_3', 'short_description', 'price', 'product_type'])
train_clarity_labels = pd.read_csv("dataset/training/clarity_train.csv", names=['clarity'])
train_conciseness_labels = pd.read_csv("dataset/training/conciseness_train.csv", names=['conciseness'])


validation_data = pd.read_csv("dataset/validation/data_valid.csv", names=['country', 'sku_id', 'title', 'category_lvl_1', 'category_lvl_2', 'category_lvl_3', 'short_description', 'price', 'product_type'])
validation_clarity_labels = pd.read_csv("dataset/validation/clarity_valid.csv", names=['clarity'])
validation_conciseness_labels = pd.read_csv("dataset/validation/conciseness_valid.csv", names=['conciseness'])


In [4]:
train_data['conciseness'] = np.loadtxt("dataset/training/conciseness_train.labels", dtype=int)
train_data['clarity'] = np.loadtxt("dataset/training/clarity_train.labels", dtype=int)

(2055, 11)

In [4]:
#train_clarity_labels


In [8]:
# train_conciseness_labels
train_data.shape

(36283, 11)

In [9]:

# preprocessing --
train_data['category_lvl_3'] = train_data['category_lvl_3'].fillna('NA')
#null values check
def null_value_chk(data):
    for column in data.columns:
        print(column, data[column].isnull().sum())


#html tags removing from description
def remove_tags(string):
    
    doc = lxml.html.fromstring(string)
    cleaner = lxml.html.clean.Cleaner(style=True)
    doc = cleaner.clean_html(doc)
    text = doc.text_content()
    return text

def clear_description(data):
    new_string = re.sub('[:!@#$]','',data )
    return new_string

#data cleaning of null vales of category

def category_data(data):
    for idx in range(0 , len(data)):
        if(pd.isnull(data['category_lvl_3'][idx]) ):
            data.loc[idx, 'category_lvl_3'] = data.loc[idx]['category_lvl_1'] + " " + data.loc[idx]['category_lvl_2']


def clean_data(data):
    
    #train_data['short_description'] = train_data['short_description'].apply(lambda x: 'no description')
    #html tags removal
    train_data['short_description'] = train_data['short_description'].apply(lambda cw : remove_tags(str(cw)))
    train_data['short_description'] = train_data['short_description'].apply(lambda x:clear_description(str(x)))


#removing description for now - will see it later
# train_data['short_description'] = train_data['short_description'].apply(lambda x: 'no description')

clean_data(train_data)
train_data.iloc[2:6]



,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type,conciseness,clarity
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,"150cm mini microphone compatible for iPhone, ...",25.07,international,0,1
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,ANMYNA Complaint Silky Set (Shampoo 520ml + C...,118.00,local,1,1
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,100% Authentic Rrefresh and brighten skin Ant...,114.80,international,1,1
5,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,Genuine Windows® 8.1 13.3in 169 HD TOUCH In...,2599.00,local,1,1


In [11]:
train_data.to_csv('train_preprocessed.csv',encoding='utf-8', index=False)


In [12]:
train_data_preprocessed = pd.read_csv("train_preprocessed.csv", encoding='utf-8')

In [87]:
colors_df = set()
brands_df = set()

def __load__():
    global colors_df
    global brands_df
    
    colors_df = set(line.strip().lower() for line in open('dataset\colors.txt'))
    brands_df = set(line.strip().lower() for line in open('dataset\lazada_brands.txt'))

__load__()

def jaccard_sim(s1, s2):
    inter = float(len(s1&s2))
    union = float(len(s1|s2))
    return inter / union  if union > 0 else -1.0

def has_numeric_value(word):
    return bool(re.search(r'\d', word))

def special_char_count(text):
    special_chars = '''!"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~ '''
    return sum(1 for c in text if c in special_chars)

def price_country(price, country):
    
    price = abs(price)
    if country == 'my':
        price *= 0.32
    elif country != 'sg':
        price *= 0.074
    
    return price


def generate_surface_features(df):
    
    feat_names = ['title_words_count', 'title_unique_words_count', 'title_duplicate_words_count', 'title_duplicate_words_ratio',\
            'title_avg_word_length','title_sku','title_has_numeric_value','title_desc_sim', 'title_cat1_sim',\
            'title_cat2_sim', 'title_cat3_sim', 'title_special_chars_count', 'price_c']
   
    feats_data = []
    
    for i, row in df.iterrows():
        
        #new features
        feat = [] 
        
        title = row['title'].lower()
        desc = row['short_description']
        sku_id = row['sku_id'].lower()
        price = float(row['price'])
        country = row['country']
        
        title_words = title.split(' ')
        title_words_count = len(title_words)
        feat.append(title_words_count)
            
        desc_words = desc.split(' ')
            
            
        #if there are many duplicate words, the title will be not concise
    
        title_unique_words = [w for w in set(title_words)]
        
        title_unique_words_count = len(title_unique_words)
        feat.append(title_unique_words_count)
        
        title_duplicate_words_count = title_words_count - title_unique_words_count
        feat.append(title_duplicate_words_count)
            
        title_duplicate_words_ratio = title_duplicate_words_count / title_words_count
        feat.append(title_duplicate_words_ratio)
        
        title_avg_word_length = len(title)/title_words_count
        feat.append(title_avg_word_length)
               
        
        # if first two characters matches with sku_id. first word in title usually shows brand
        title_sku = int(title[:2] == sku_id[:2])
        feat.append(title_sku)
            
        #contains some numeric value which may show product model
        title_has_numeric_value = int(any([ has_numeric_value(w) for w in title_words ]))
        feat.append(title_has_numeric_value)
        
        #jaccard similarity score between title and description
        title_desc_sim = jaccard_sim(set(title_words),set(desc_words))
        feat.append(title_desc_sim)
        
        
        
        cat1 = row['category_lvl_1'].lower().split()
        cat2 = row['category_lvl_2'].lower().split()
        cat3 = row['category_lvl_3'].lower().split()
        
        # similarity between title words and category words
        feat.append(len(set(title_words)&set(cat1)))
        feat.append(len(set(title_words)&set(cat2)))
        feat.append(len(set(title_words)&set(cat3)))
        
        
        #special character count
        title_special_chars_count = special_char_count(title)
        feat.append(title_special_chars_count)
        
        #price country
        price_c = price_country(price,country)
        feat.append(price_c) 
        
        feats_data.append(feat)
        
    
    return pd.DataFrame(feats_data, columns=feat_names)
    #return scipy.sparse.csr_matrix(feats_data)
        
train_data_feats = generate_surface_features(train_data)


In [75]:
def generate_title_text_features(train_data,validation_data,analyzer="char", n_grams=(2,6), min_df = 1, max_df = 1.0):
    #title text features ngrams
    
    n_tr = train_data.shape[0]
    
    
    df = pd.concat((train_data,validation_data))
    
    countVectorizer = CountVectorizer(analyzer=analyzer, ngram_range=n_grams, min_df=min_df, max_df = max_df, lowercase=True)
    countVectorizer.fit(df['title'])
    X = countVectorizer.transform(df['title'].values)
    
    X_tr = X[:n_tr,:]
    X_val = X[n_tr:,:]
    
    #print(countVectorizer.get_feature_names())
    with open("features/title_ng_tr.pickle","wb") as f:
        pickle.dump(X_tr, f)
    
    with open("features/title_ng_val.pickle","wb") as f:
        pickle.dump(X_val, f)
        
    return X_tr,X_val
  
X_title_count_tr,X_title_count_val = generate_title_text_features(train_data,validation_data, min_df = 0.005)
# train_data_feats['title_text_count_feat'] = generate_title_text_features(train_data)
X_title_count_tr.shape

In [86]:
def extract_brand_features(train_data, validation_data, analyzer='word', ngram_range=(1, 1), lowercase=True):
    
    n_trains = train_data.shape[0]
    
    df = pd.concat((train_data,validation_data))
    
    brands = [x.strip() for x in open("dataset/lazada_brands.txt").readlines()]
    vect = TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range, decode_error='replace')
    vect.fit(brands)
    X = vect.transform(df["title"].map(lambda x: x.replace('(', ' (').replace(')', ' )').lower()).values)
    X_tr = X[:n_trains,:]
    X_val = X[n_trains:,:]
    feat_train_file = "title_brands.trn"
    feat_test_file = "title_brands.tst"

    with open("features/brands_feats_tr.pickle","wb") as f:
        pickle.dump(X_tr, f)
    
    with open("features/brands_feats_val.pickle","wb") as f:
        pickle.dump(X_val, f)

extract_brand_features(train_data,validation_data)  

In [72]:
def extract_color_features(train_data, validation_data, analyzer='word', ngram_range=(1, 1), lowercase=True):
    
    n_trains = train_data.shape[0]
    
    df = pd.concat((train_data,validation_data))
    
    colors = [x.strip() for x in open("dataset\colors.txt").readlines()]
    
    vect = TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range, decode_error='replace')
    vect.fit(colors)
    
    X = vect.transform(df["title"].map(lambda x: x.replace('(', ' (').replace(')', ' )').lower()).values)
    X_tr = X[:n_trains,:]
    X_val = X[n_trains:,:]
    feat_train_file = "title_color.trn"
    feat_test_file = "title_color.tst"

    with open("features/color_feats_tr.pickle","wb") as f:
        pickle.dump(X_tr, f)
    
    with open("features/color_feats_val.pickle","wb") as f:
        pickle.dump(X_val, f)

extract_color_features(train_data,validation_data)  

In [90]:
with open("features/title_ng_tr.pickle","rb") as f:
    X_title_count_tr = pickle.load(f)
    
with open("features/title_ng_val.pickle","rb") as f:
    X_title_count_val = pickle.load(f)
    
with open("features/color_feats_tr.pickle","rb") as f:
    X_title_color_tr = pickle.load(f)
    
with open("features/color_feats_val.pickle","rb") as f:
    X_title_color_val = pickle.load(f)

with open("features/brands_feats_tr.pickle","rb") as f:
    X_title_brands_tr = pickle.load(f)
    
with open("features/brands_feats_val.pickle","rb") as f:
    X_title_brands_val = pickle.load(f)

In [91]:
X_surf_feats_tr = scipy.sparse.csr_matrix(train_data_feats.to_sparse())

In [92]:
X_feats_tr = hstack((X_title_count_tr,X_title_color_tr,X_title_brands_tr, X_surf_feats_tr))

In [99]:
X_feats_tr = X_feats_tr.tocsr()

In [100]:
X_feats_tr

<36283x28729 sparse matrix of type '<class 'numpy.float64'>'
	with 7858668 stored elements in Compressed Sparse Row format>

In [24]:
np.array(validation_clarity_labels['clarity'])


array([0.34139, 0.22758, 0.36162, ..., 0.39207, 0.48232, 0.27523])

In [25]:
X_title_count_tr = X_title_count_tr.astype('float32')


In [110]:
X_title_count_tr

<36283x8768 sparse matrix of type '<class 'numpy.int64'>'
	with 7261026 stored elements in Compressed Sparse Row format>

In [109]:
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True)
lgb_model = lgb.LGBMClassifier()
YStrat = train_data['conciseness'].values
step = 1
X = X_feats_tr
avg_rmse = 0
for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = train_data['conciseness'][train_index]
    test_X = X[test_index]
    test_Y = train_data['conciseness'][test_index]
    lgb_model.fit(train_X, train_Y)
    predictions = lgb_model.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    mse = mean_squared_error(test_Y, predictions)
    rmse = math.sqrt(mse)
    avg_rmse += mse
    print("LGB model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("LGB model rmse score for K fold {} = {}".format(step, rmse) )
    step += 1

avg_rmse = math.sqrt(avg_rmse/k)
print("AVG RMSE: ", avg_rmse)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 1 = 0.8495453292918159
LGB model rmse score for K fold 1 = 0.3878848678515109


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 2 = 0.8525764673463764
LGB model rmse score for K fold 2 = 0.38395772248207694


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 8 = 0.8524951750758203
LGB model rmse score for K fold 8 = 0.38406356885830734


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 9 = 0.8417424869037772
LGB model rmse score for K fold 9 = 0.3978159286607598
LGB model accuracy score for K fold 10 = 0.8475323959194927
LGB model rmse score for K fold 10 = 0.3904710028676999
AVG RMSE:  0.38941158787988156


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [98]:
X_feats_tr


<36283x28729 sparse matrix of type '<class 'numpy.float64'>'
	with 7858668 stored elements in COOrdinate format>

In [101]:
kf = StratifiedKFold(n_splits=10, shuffle=True)
lgb_model = lgb.LGBMClassifier()
YStrat = train_data['clarity'].values
step = 1
X = X_feats_tr

for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = train_data['clarity'][train_index]
    test_X = X[test_index]
    test_Y = train_data['clarity'][test_index]
    lgb_model.fit(train_X, train_Y)
    predictions = lgb_model.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    rmse = math.sqrt(mean_squared_error(test_Y, predictions))
    print("LGB model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("LGB model rmse score for K fold {} = {}".format(step, rmse) )
    step += 1

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 1 = 0.9421328189583907
LGB model rmse score for K fold 1 = 0.24055598317566176


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 2 = 0.9382750068889502
LGB model rmse score for K fold 2 = 0.24844515111197055


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 8 = 0.9445975744211687
LGB model rmse score for K fold 8 = 0.23537719851088235


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGB model accuracy score for K fold 9 = 0.941825199889716
LGB model rmse score for K fold 9 = 0.24119452752971818
LGB model accuracy score for K fold 10 = 0.9423766197959746
LGB model rmse score for K fold 10 = 0.24004870381659088


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [108]:
# title ngrams used only for NaiveBayesian. Giving huge errors for more features.

kf = StratifiedKFold(n_splits=10, shuffle=True)
naive_bayes_model = MultinomialNB()
YStrat = train_data['conciseness'].values
step = 1
X = X_title_count_tr
for train_index, test_index in kf.split(X,YStrat): 
    print("train_index", train_index)
    train_X = X[train_index]
    train_Y = train_data['conciseness'][train_index]
    test_X = X[test_index]
    test_Y = train_data['conciseness'][test_index]
    naive_bayes_model.fit(train_X, train_Y)
    predictions = naive_bayes_model.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    rmse = math.sqrt(mean_squared_error(test_Y, predictions))
    print("Naive Bayesian model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("Naive Bayesian model rmse score for K fold {} = {}".format(step, rmse) )


    step += 1

train_index [    1     2     3 ... 36279 36280 36281]
Naive Bayesian model accuracy score for K fold 1 = 0.43042160374758887
Naive Bayesian model rmse score for K fold 1 = 0.7547041779746626
train_index [    0     1     3 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 2 = 0.42270597960870765
Naive Bayesian model rmse score for K fold 2 = 0.7597986709591511
train_index [    0     2     3 ... 36278 36279 36282]
Naive Bayesian model accuracy score for K fold 3 = 0.43152383576742903
Naive Bayesian model rmse score for K fold 3 = 0.7539735832458395
train_index [    0     1     2 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 4 = 0.42904381372278866
Naive Bayesian model rmse score for K fold 4 = 0.7556164280090867
train_index [    0     1     2 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 5 = 0.4262882336731882
Naive Bayesian model rmse score for K fold 5 = 0.757437631971644
train_index [    0     1     2 ... 36280 36281 

In [107]:
# title ngrams used only for NaiveBayesian. Giving huge errors for more features.

kf = StratifiedKFold(n_splits=10, shuffle=True)
naive_bayes_model = MultinomialNB()
YStrat = train_data['clarity'].values
step = 1
X = X_title_count_tr
for train_index, test_index in kf.split(X,YStrat): 
    print("train_index", train_index)
    train_X = X[train_index]
    train_Y = train_data['clarity'][train_index]
    test_X = X[test_index]
    test_Y = train_data['clarity'][test_index]
    naive_bayes_model.fit(train_X, train_Y)
    predictions = naive_bayes_model.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    rmse = math.sqrt(mean_squared_error(test_Y, predictions))
    print("Naive Bayesian model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("Naive Bayesian model rmse score for K fold {} = {}".format(step, rmse) )

    step += 1

train_index [    1     2     4 ... 36278 36280 36282]
Naive Bayesian model accuracy score for K fold 1 = 0.7431799393772389
Naive Bayesian model rmse score for K fold 1 = 0.506774171226949
train_index [    0     3     4 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 2 = 0.7503444475062001
Naive Bayesian model rmse score for K fold 2 = 0.49965543376791166
train_index [    0     1     2 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 3 = 0.7426288233673188
Naive Bayesian model rmse score for K fold 3 = 0.5073176289393866
train_index [    0     1     2 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 4 = 0.7456599614218793
Naive Bayesian model rmse score for K fold 4 = 0.5043213643879473
train_index [    0     1     2 ... 36280 36281 36282]
Naive Bayesian model accuracy score for K fold 5 = 0.7591623036649214
Naive Bayesian model rmse score for K fold 5 = 0.49075217405028226
train_index [    0     1     2 ... 36280 36281 36

In [32]:
X_feats_tr_csr

<36283x8790 sparse matrix of type '<class 'numpy.float64'>'
	with 7578925 stored elements in Compressed Sparse Row format>

In [156]:
X_title_count_tr.astype('float32')

<36283x8775 sparse matrix of type '<class 'numpy.float32'>'
	with 7263264 stored elements in Compressed Sparse Row format>

In [138]:
train_conciseness_labels[train_conciseness_labels['conciseness'] == 0]




,conciseness
2,0
18,0
23,0
36,0
38,0
41,0
42,0
43,0
44,0
45,0
